# **SQL Flight Booking Analysis Project**



Project: Flight Booking Sales Analysis & Forecasting using SQL
Tools: SQLite, Colab, Pandas

Description:
Analyzed a large dataset of flight bookings to identify route popularity, customer preferences, booking lead times, and monthly sales trends. Developed SQL queries to forecast future bookings based on historical growth patterns and customer behavior.

In [3]:
from google.colab import files
uploaded=files.upload()


Saving customer_booking.csv to customer_booking.csv


In [4]:
import pandas as pd
df=pd.read_csv('customer_booking.csv', encoding='latin-1')
df.head(10)

,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
0,2,Internet,RoundTrip,262,19,7,Sat,AKLDEL,New Zealand,1,0,0,5.52,0
1,1,Internet,RoundTrip,112,20,3,Sat,AKLDEL,New Zealand,0,0,0,5.52,0
2,2,Internet,RoundTrip,243,22,17,Wed,AKLDEL,India,1,1,0,5.52,0
3,1,Internet,RoundTrip,96,31,4,Sat,AKLDEL,New Zealand,0,0,1,5.52,0
4,2,Internet,RoundTrip,68,22,15,Wed,AKLDEL,India,1,0,1,5.52,0
5,1,Internet,RoundTrip,3,48,20,Thu,AKLDEL,New Zealand,1,0,1,5.52,0
6,3,Internet,RoundTrip,201,33,6,Thu,AKLDEL,New Zealand,1,0,1,5.52,0
7,2,Internet,RoundTrip,238,19,14,Mon,AKLDEL,India,1,0,1,5.52,0
8,1,Internet,RoundTrip,80,22,4,Mon,AKLDEL,New Zealand,0,0,1,5.52,0
9,1,Mobile,RoundTrip,378,30,12,Sun,AKLDEL,India,0,0,0,5.52,0


In [5]:
import sqlite3

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Write the dataframe to SQL table
df.to_sql('my_table', conn, index=False, if_exists='replace')


50000

In [6]:

# Run SQL queries
query = "SELECT * FROM my_table LIMIT 5;"
result = pd.read_sql_query(query, conn)
result


,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
0,2,Internet,RoundTrip,262,19,7,Sat,AKLDEL,New Zealand,1,0,0,5.52,0
1,1,Internet,RoundTrip,112,20,3,Sat,AKLDEL,New Zealand,0,0,0,5.52,0
2,2,Internet,RoundTrip,243,22,17,Wed,AKLDEL,India,1,1,0,5.52,0
3,1,Internet,RoundTrip,96,31,4,Sat,AKLDEL,New Zealand,0,0,1,5.52,0
4,2,Internet,RoundTrip,68,22,15,Wed,AKLDEL,India,1,0,1,5.52,0


Create a Table (Optional, if you don't already have CSV data)**bold text**

In [19]:
import sqlite3

conn = sqlite3.connect('test.db')
print("Opened database successfully");
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE flight_data (
    num_passengers INTEGER,
    sales_channel TEXT,
    trip_type TEXT,
    purchase_lead INTEGER,
    length_of_stay INTEGER,
    flight_hour INTEGER,
    flight_day TEXT,
    route TEXT,
    booking_origin TEXT,
    wants_extra_baggage BOOLEAN,
    wants_preferred_seat BOOLEAN,
    wants_in_flight_meals BOOLEAN,
    flight_duration REAL,
    booking_complete BOOLEAN
);
''')

print("Table created successfully")


In [24]:
query = """
SELECT route, COUNT(*) AS total_bookings
FROM my_table
WHERE booking_complete = 1
GROUP BY route
ORDER BY total_bookings DESC
LIMIT 5;
"""

df_result = pd.read_sql_query(query, conn)
df_result


,route,total_bookings
0,AKLKUL,567
1,PENTPE,401
2,DMKKIX,187
3,JHBKTM,164
4,MELPEN,137


In [25]:
query= """SELECT trip_type, AVG(flight_duration) AS avg_duration
FROM my_table
GROUP BY trip_type;"""
df_result = pd.read_sql_query(query, conn)
df_result

,trip_type,avg_duration
0,CircleTrip,7.764828
1,OneWay,7.966770
2,RoundTrip,7.271030


# Q1: What are the top 3 most popular flight routes?**bold text**

In [30]:
query="""SELECT route, COUNT(*) AS total_bookings
FROM  my_table
GROUP BY route
ORDER BY total_bookings DESC
LIMIT 6;
"""
df_result = pd.read_sql_query(query, conn)
df_result

,route,total_bookings
0,AKLKUL,2680
1,PENTPE,924
2,MELSGN,842
3,ICNSIN,801
4,DMKKIX,744
5,ICNSYD,695


# **Q2: What is the average number of passengers per booking?**

In [8]:
query="""SELECT AVG(num_passengers) AS avg_passengers_per_booking
FROM my_table;
"""
df_result=pd.read_sql(query,conn)
df_result

,avg_passengers_per_booking
0,1.59124


# **Q3: Which sales channel is used more frequently?**

In [10]:
query=""" SELECT sales_channel, COUNT() AS frequency
FROM my_table
GROUP BY sales_channel
ORDER BY frequency DESC
LIMIT 2;"""
df_result=pd.read_sql(query,conn)
print(df_result)

  sales_channel  frequency
0      Internet      44382
1        Mobile       5618


# **Do most users prefer extra baggage?**

In [12]:
query="""SELECT wants_extra_baggage, COUNT(*) AS count
FROM my_table
GROUP BY wants_extra_baggage;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

   wants_extra_baggage  count
0                    0  16561
1                    1  33439


# **Q5: What is the average purchase lead time for completed bookings?**

In [13]:
query="""SELECT AVG(purchase_lead) AS avg_lead_days
FROM my_table
WHERE booking_complete = 1;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

   avg_lead_days
0      80.167157


# What’s the average flight duration per route?**bold text**

In [14]:
query="""SELECT route, AVG(flight_duration) AS avg_duration
FROM my_table
GROUP BY route
ORDER BY avg_duration DESC;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

      route  avg_duration
0    JEDSUB          9.50
1    JEDPEN          9.50
2    JEDPDG          9.50
3    JEDMNL          9.50
4    JEDMFM          9.50
..      ...           ...
794  AKLTPE          4.67
795  JOGTPE          4.67
796  CNXTPE          4.67
797  DPSTPE          4.67
798  KNOTPE          4.67

[799 rows x 2 columns]


# **Q7: Which day of the week has the highest number of flights?**

In [15]:
query="""SELECT flight_day, COUNT(*) AS total_flights
FROM my_table
GROUP BY flight_day
ORDER BY total_flights DESC;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

  flight_day  total_flights
0        Mon           8102
1        Wed           7674
2        Tue           7673
3        Thu           7424
4        Fri           6761
5        Sun           6554
6        Sat           5812


# Q8: How many passengers opted for all services (baggage, seat, meals)?**bold text**

In [17]:
query="""SELECT COUNT(*) AS passengers_with_all_services
FROM my_table
WHERE wants_extra_baggage = 1
  AND wants_preferred_seat = 1
  AND wants_in_flight_meals = 1;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

   passengers_with_all_services
0                          8770


# **Q9: What's the percentage of completed bookings?bold text**

In [19]:
query="""SELECT
  (COUNT(*) * 100.0 /
    (SELECT COUNT(*) FROM my_table)) AS booking_completion_rate
FROM my_table
WHERE booking_complete = 1;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

   booking_completion_rate
0                   14.956


# Q10: Which booking origin has the highest number of completed bookings?**bold text**

In [20]:
query="""SELECT booking_origin, COUNT(*) AS completed_bookings
FROM my_table
WHERE booking_complete = 1
GROUP BY booking_origin
ORDER BY completed_bookings DESC;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

          booking_origin  completed_bookings
0               Malaysia                2468
1              Australia                 900
2                  China                 694
3              Indonesia                 615
4                  Japan                 478
5               Thailand                 470
6            South Korea                 462
7              Singapore                 299
8                 Taiwan                 224
9                  India                 131
10               Vietnam                 114
11                 Macau                  95
12         United States                  84
13           Philippines                  75
14             Hong Kong                  73
15           New Zealand                  54
16                Brunei                  41
17              Cambodia                  36
18        United Kingdom                  21
19                France                  14
20             (not set)                  14
21       M

## SQL Queries for Future Sales Prediction (Trend-Based)

# 1. Trend: Monthly Booking Growth

In [22]:
query="""
SELECT
  STRFTIME('%Y-%m', DATE('now', '-' || purchase_lead || ' days')) AS booking_month,
  COUNT(*) AS total_bookings
FROM my_table
GROUP BY booking_month
ORDER BY booking_month;
"""

df_result=pd.read_sql(query,conn)
print(df_result)

   booking_month  total_bookings
0        2022-11               1
1        2023-05               1
2        2023-07               3
3        2023-08               3
4        2023-09               4
5        2023-10               4
6        2023-11               2
7        2023-12               4
8        2024-01              23
9        2024-02             212
10       2024-03             411
11       2024-04             441
12       2024-05             629
13       2024-06             726
14       2024-07             954
15       2024-08            1435
16       2024-09            1481
17       2024-10            1891
18       2024-11            2167
19       2024-12            3523
20       2025-01            5227
21       2025-02            7877
22       2025-03           14126
23       2025-04            8855


# **2. Average Purchase Lead Time per Month**

In [23]:
query="""SELECT
  STRFTIME('%Y-%m', DATE('now', '-' || purchase_lead || ' days')) AS booking_month,
  AVG(purchase_lead) AS avg_lead_days
FROM my_table
GROUP BY booking_month
ORDER BY booking_month;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

   booking_month  avg_lead_days
0        2022-11     867.000000
1        2023-05     704.000000
2        2023-07     633.000000
3        2023-08     608.333333
4        2023-09     574.500000
5        2023-10     548.500000
6        2023-11     515.000000
7        2023-12     481.500000
8        2024-01     445.173913
9        2024-02     421.811321
10       2024-03     394.403893
11       2024-04     362.587302
12       2024-05     333.303657
13       2024-06     302.227273
14       2024-07     272.136268
15       2024-08     241.347038
16       2024-09     211.253883
17       2024-10     180.038604
18       2024-11     150.242270
19       2024-12     118.744820
20       2025-01      87.260761
21       2025-02      58.102196
22       2025-03      28.643140
23       2025-04       6.998532


# 3. Top 5 Routes (for future marketing focus)

In [7]:
query="""SELECT
  route,
  COUNT(*) AS total_bookings
FROM my_table
WHERE booking_complete = 1
GROUP BY route
ORDER BY total_bookings DESC
LIMIT 5;
"""

df_result=pd.read_sql(query,conn)
print(df_result)

    route  total_bookings
0  AKLKUL             567
1  PENTPE             401
2  DMKKIX             187
3  JHBKTM             164
4  MELPEN             137


# 4. Most Preferred Flight Days**

# **5. Most Common Add-on Combinations**

In [8]:
query="""SELECT
  wants_extra_baggage,
  wants_preferred_seat,
  wants_in_flight_meals,
  COUNT(*) AS count
FROM my_table
GROUP BY wants_extra_baggage, wants_preferred_seat, wants_in_flight_meals
ORDER BY count DESC;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

   wants_extra_baggage  wants_preferred_seat  wants_in_flight_meals  count
0                    1                     0                      0  13243
1                    0                     0                      0  10455
2                    1                     1                      1   8770
3                    1                     0                      1   8033
4                    0                     0                      1   3421
5                    1                     1                      0   3393
6                    0                     1                      0   1552
7                    0                     1                      1   1133


# **6. Predicting Growth Rate in Bookings**

In [9]:
query="""WITH monthly_bookings AS (
  SELECT
    STRFTIME('%Y-%m', DATE('now', '-' || purchase_lead || ' days')) AS month,
    COUNT(*) AS bookings
  FROM my_table
  GROUP BY month
),
growth AS (
  SELECT
    month,
    bookings,
    LAG(bookings) OVER (ORDER BY month) AS prev_bookings
  FROM monthly_bookings
)
SELECT
  month,
  bookings,
  prev_bookings,
  ROUND(((bookings - prev_bookings) * 100.0) / prev_bookings, 2) AS growth_percent
FROM growth
WHERE prev_bookings IS NOT NULL;
"""
df_result=pd.read_sql(query,conn)
print(df_result)

      month  bookings  prev_bookings  growth_percent
0   2023-05         1              1            0.00
1   2023-07         3              1          200.00
2   2023-08         3              3            0.00
3   2023-09         4              3           33.33
4   2023-10         4              4            0.00
5   2023-11         2              4          -50.00
6   2023-12         3              2           50.00
7   2024-01        19              3          533.33
8   2024-02       211             19         1010.53
9   2024-03       396            211           87.68
10  2024-04       445            396           12.37
11  2024-05       632            445           42.02
12  2024-06       718            632           13.61
13  2024-07       939            718           30.78
14  2024-08      1421            939           51.33
15  2024-09      1486           1421            4.57
16  2024-10      1880           1486           26.51
17  2024-11      2152           1880          